## Import libraries:


In [1]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1620544976176_0008,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.appName("als1").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Load Movie Ratings Data

In [3]:
# Load the ratings data into a dataframe
ratings = spark.read.csv("s3a://sparkdemonstration-mlc/ratings.csv",inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Print the ratings dataframe
ratings.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows

In [5]:
# Drop the timestamp column from the ratings dataframe.
ratings=ratings.drop("timestamp")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Print the rating dataframe after dropping the timestamp column.
ratings.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    296|   5.0|
|     1|    306|   3.5|
|     1|    307|   5.0|
|     1|    665|   5.0|
|     1|    899|   3.5|
|     1|   1088|   4.0|
|     1|   1175|   3.5|
|     1|   1217|   3.5|
|     1|   1237|   5.0|
|     1|   1250|   4.0|
|     1|   1260|   3.5|
|     1|   1653|   4.0|
|     1|   2011|   2.5|
|     1|   2012|   2.5|
|     1|   2068|   2.5|
|     1|   2161|   3.5|
|     1|   2351|   4.5|
|     1|   2573|   4.0|
|     1|   2632|   5.0|
|     1|   2692|   5.0|
+------+-------+------+
only showing top 20 rows

## Load New User Data

In [7]:
# Load the New user's ratings data into a dataframe New_user_rating
New_user_rating = spark.read.csv("s3a://sparkdemonstration-mlc/New_user_rating.csv",inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Concatenate the Movie Rating data with new user Data.

In [8]:
# Concatenate the ratings and New_user_rating dataframes into single dataframe.
rating_final = ratings.union(New_user_rating)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Describe the "rating_final" dataframe.
rating_final.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|    296|   5.0|
|     1|    306|   3.5|
|     1|    307|   5.0|
|     1|    665|   5.0|
|     1|    899|   3.5|
|     1|   1088|   4.0|
|     1|   1175|   3.5|
|     1|   1217|   3.5|
|     1|   1237|   5.0|
|     1|   1250|   4.0|
|     1|   1260|   3.5|
|     1|   1653|   4.0|
|     1|   2011|   2.5|
|     1|   2012|   2.5|
|     1|   2068|   2.5|
|     1|   2161|   3.5|
|     1|   2351|   4.5|
|     1|   2573|   4.0|
|     1|   2632|   5.0|
|     1|   2692|   5.0|
+------+-------+------+
only showing top 20 rows

In [10]:
rating_final.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

25000145

In [21]:
rating_final.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+-----------------+------------------+
|summary|           userId|          movieId|            rating|
+-------+-----------------+-----------------+------------------+
|  count|         25000145|         25000145|          25000145|
|   mean|81189.44385830562|21388.09762391378|3.5338546436430667|
| stddev|46791.81054576984|39198.98818722356|1.0607436520838271|
|    min|                1|                1|               0.5|
|    max|           162542|           209171|               5.0|
+-------+-----------------+-----------------+------------------+

## Load Movies Data

In [11]:
# Load the Movies data into a dataframe
movies = spark.read.csv("s3a://sparkdemonstration-mlc/movies.csv",inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Print the Movies data.
movies.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

## Split the Rating data to Train & Test set

In [17]:
# Split the "rating_final" data into training and test data.
(training, validation, test) = rating_final.randomSplit([0.6, 0.2, 0.2], seed=99)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## ALS Model Building with iteration value = 3

In [14]:
# Build the ALS model.
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",maxIter=3, regParam=0.05, coldStartStrategy="drop")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
model=als.fit(training)
prediction=model.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
prediction.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|114572|    148|   2.0| 2.0873017|
|124207|    148|   3.0| 1.4196813|
|159730|    148|   3.0| 2.8212676|
| 99010|    148|   3.0| 2.3998055|
| 57779|    148|   2.0| 2.1616316|
|  6491|    148|   4.0| 2.2291267|
| 14831|    148|   3.0| 2.4541855|
|145182|    148|   3.0| 2.7872567|
| 69123|    148|   4.5|   2.09973|
| 84667|    148|   5.0| 3.0733442|
|162516|    148|   2.0| 1.9329386|
| 73977|    148|   2.0| 3.2117496|
| 75209|    148|   2.0| 1.4785355|
|149570|    148|   1.0| 1.9827228|
|109156|    148|   2.0| 2.5137262|
| 19478|    148|   2.0| 2.5636063|
| 58005|    148|   3.0| 2.4552035|
|110863|    148|   3.0| 2.7237966|
|  2545|    148|   5.0| 3.2926793|
| 69855|    148|   4.0| 3.0395195|
+------+-------+------+----------+
only showing top 20 rows

In [20]:
# Print the RMSE value.
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(prediction) 
rmse

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8365479217327022

## Recommend the top 20 movies to the new user with user id = 162542

In [22]:
# Get the top 20 movie recommendations for all the users.
userRecs = model.recommendForAllUsers(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Print the userRecs.
userRecs.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   148|[[159467, 10.6528...|
|   463|[[205277, 13.4987...|
|   471|[[205277, 11.3107...|
|   496|[[159467, 12.8232...|
|   833|[[205277, 9.64507...|
|  1088|[[192689, 11.1319...|
|  1238|[[205277, 11.7469...|
|  1342|[[159467, 10.1689...|
|  1580|[[205277, 11.4177...|
|  1591|[[205277, 14.1690...|
|  1645|[[205277, 10.8278...|
|  1829|[[205277, 10.5792...|
|  1959|[[205277, 12.3869...|
|  2122|[[126941, 9.51770...|
|  2142|[[205277, 11.1007...|
|  2366|[[159467, 10.8585...|
|  2659|[[151410, 11.4519...|
|  2866|[[202231, 10.9826...|
|  3175|[[205277, 10.8022...|
|  3749|[[187999, 9.77840...|
+------+--------------------+
only showing top 20 rows

In [24]:
# Filter the recommendations of top 10 movies for userId 162542 from "userRecs".
df= userRecs.where(userRecs.userId == 162542).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
df.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'list' object has no attribute 'dtypes'
Traceback (most recent call last):
AttributeError: 'list' object has no attribute 'dtypes'



In [27]:
# Convert the 'df' object into the dataframe.
df = spark.createDataFrame(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# Print the df dataframe.
df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(userId=162542, recommendations=[Row(movieId=205277, rating=9.24085521697998), Row(movieId=174529, rating=9.075715065002441), Row(movieId=194160, rating=7.9748358726501465), Row(movieId=26968, rating=7.767980098724365), Row(movieId=202231, rating=7.633571624755859), Row(movieId=126941, rating=7.5803046226501465), Row(movieId=179979, rating=7.420927047729492), Row(movieId=194202, rating=7.349876403808594), Row(movieId=176235, rating=7.2541656494140625), Row(movieId=36816, rating=7.252978324890137), Row(movieId=101954, rating=7.151526927947998), Row(movieId=185655, rating=7.103945732116699), Row(movieId=171873, rating=6.98195219039917), Row(movieId=184891, rating=6.979526042938232), Row(movieId=92516, rating=6.920289516448975), Row(movieId=206911, rating=6.917118549346924), Row(movieId=134037, rating=6.902005672454834), Row(movieId=142871, rating=6.900271415710449), Row(movieId=175625, rating=6.872191429138184), Row(movieId=155042, rating=6.864363670349121)])

In [29]:
# Get the recommended movies for the userId 162542 into df1 list.
df1 = df.head()[1]

#convert the df1 into dataframe.
df1= spark.createDataFrame(df1)
df1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|movieId|            rating|
+-------+------------------+
| 205277|  9.24085521697998|
| 174529| 9.075715065002441|
| 194160|7.9748358726501465|
|  26968| 7.767980098724365|
| 202231| 7.633571624755859|
| 126941|7.5803046226501465|
| 179979| 7.420927047729492|
| 194202| 7.349876403808594|
| 176235|7.2541656494140625|
|  36816| 7.252978324890137|
| 101954| 7.151526927947998|
| 185655| 7.103945732116699|
| 171873|  6.98195219039917|
| 184891| 6.979526042938232|
|  92516| 6.920289516448975|
| 206911| 6.917118549346924|
| 134037| 6.902005672454834|
| 142871| 6.900271415710449|
| 175625| 6.872191429138184|
| 155042| 6.864363670349121|
+-------+------------------+

In [30]:
# Join the df1 dataframe with movies data to see the name of the recommended movies.
userRecs_joined = df1.join(movies,on='movieId',how="left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# Print the recommended movies for the new user with userId 162542.
userRecs_joined.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+--------------------+--------------------+
|movieId|            rating|               title|              genres|
+-------+------------------+--------------------+--------------------+
| 205277|  9.24085521697998|   Inside Out (1991)|Comedy|Drama|Romance|
| 174529| 9.075715065002441|The Easter Bunny ...|Animation|Childre...|
| 194160|7.9748358726501465|  The Carrier (1988)|Drama|Horror|Thri...|
|  26968| 7.767980098724365|  Cremaster 5 (1997)|       Drama|Musical|
| 202231| 7.633571624755859|       Foster (2018)|         Documentary|
| 126941|7.5803046226501465|Joni's Promise (2...|              Comedy|
| 179979| 7.420927047729492|Heroes Above All ...|  Adventure|Children|
| 194202| 7.349876403808594|     Betrayed (2018)|  (no genres listed)|
| 176235|7.2541656494140625|Pooh's Heffalump ...|  Animation|Children|
|  36816| 7.252978324890137|Angel in My Pocke...|              Comedy|
| 101954| 7.151526927947998|Alex in Wonderlan...|        Comedy|Drama|
| 1856